In [2]:
from pathlib import Path
import json
import shutil
from glob import glob
from typing import Any, Dict, List, Optional, Set, Tuple

from PIL import Image

from tqdm import tqdm

In [3]:
# 학습(train) 세트 경로
TRAIN_ROOT: Path = Path("dataset/traffic_data/training/bbox_tr")
TRAIN_LABEL_DIR: Path = TRAIN_ROOT / "label_bbox"
TRAIN_IMAGE_DIR: Path = TRAIN_ROOT / "image_bbox"

# 검증(val) 세트 경로
VAL_ROOT: Path = Path("dataset/traffic_data/validation/bbox_val")
VAL_LABEL_DIR: Path = VAL_ROOT / "label_bbox"
VAL_IMAGE_DIR: Path = VAL_ROOT / "image_bbox"

# 출력 루트
OUT_ROOT: Path = Path("yolo_transfer")

# 전이학습 3단계 규칙 -> 정규화 없이 값 그대로 비교
STAGES: Dict[str, Dict[str, Set[str]]] = {
    "day_data": {
        "weathers": set(["Sunny"]),
        "times": set(["낮", "오전"])
    },
    "night_data": {
        "weathers": set(["Sunny"]),
        "times": set(["새벽", "오후"])
    },
    "bad_weather_data": {
        "weathers": set(["Cloudy", "Foggy", "Rainy", "Snow"]),
        "times": set(["낮", "오전", "새벽", "오후"])
    }
}

# 실행 옵션
OVERWRITE_LABELS: bool = True
COPY_IMAGES: bool = True
DRY_RUN: bool = False


In [4]:
def ensure_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)

def build_camera_index(images_root: Path) -> Dict[str, List[Path]]:
    # 패턴 일반화 -> image_bbox/site1/site2/camera_id/ 까지 모든 디렉터리 순회
    cam_index: Dict[str, List[Path]] = {}
    if not images_root.exists():
        return cam_index
    site1: Path
    for site1 in images_root.iterdir():
        if not site1.is_dir():
            continue
        site2: Path
        for site2 in site1.iterdir():
            if not site2.is_dir():
                continue
            cam_dir: Path
            for cam_dir in site2.iterdir():
                if not cam_dir.is_dir():
                    continue
                cam_id: str = cam_dir.name
                cam_index.setdefault(cam_id, []).append(cam_dir)
    return cam_index

def find_image_path_from_index(cam_index: Dict[str, List[Path]], camera_id: str, filename: str) -> Optional[Path]:
    dirs: List[Path] = cam_index.get(camera_id, [])
    d: Path
    for d in dirs:
        candidate: Path = d / filename
        if candidate.exists():
            return candidate
    return None

def xyxy_to_yolo(x1: float, y1: float, x2: float, y2: float, w: int, h: int) -> Tuple[float, float, float, float]:
    x1c: float = float(min(max(x1, 0.0), float(w)))
    y1c: float = float(min(max(y1, 0.0), float(h)))
    x2c: float = float(min(max(x2, 0.0), float(w)))
    y2c: float = float(min(max(y2, 0.0), float(h)))

    xmin: float = float(min(x1c, x2c))
    xmax: float = float(max(x1c, x2c))
    ymin: float = float(min(y1c, y2c))
    ymax: float = float(max(y1c, y2c))

    bw: float = float(max(xmax - xmin, 1e-6))
    bh: float = float(max(ymax - ymin, 1e-6))

    cx: float = float(xmin + bw / 2.0)
    cy: float = float(ymin + bh / 2.0)

    x_norm: float = float(cx / float(w))
    y_norm: float = float(cy / float(h))
    w_norm: float = float(bw / float(w))
    h_norm: float = float(bh / float(h))
    return x_norm, y_norm, w_norm, h_norm


In [5]:
def decide_stage(weather: str, time_space: str, stages: Dict[str, Dict[str, Set[str]]]) -> Optional[str]:
    stage_name: str
    rule: Dict[str, Set[str]]
    for stage_name, rule in stages.items():
        if weather in rule["weathers"] and time_space in rule["times"]:
            return stage_name
    return None


In [6]:
def process_coco_json_fixed_subset(
    json_path: Path,
    cam_index: Dict[str, List[Path]],
    out_root: Path,
    stages: Dict[str, Dict[str, Set[str]]],
    id_to_idx: Dict[int, int],
    id_to_name: Dict[int, str],
    overwrite_labels: bool,
    copy_images: bool,
    dry_run: bool,
    subset_name: str
) -> Dict[str, int]:
    counts: Dict[str, int] = {"copied": 0, "labeled": 0, "missing_image": 0, "skipped_stage": 0}

    with open(json_path, "r", encoding="utf-8") as f:
        coco_any: Any = json.load(f)

    meta_list: List[Dict[str, Any]] = list(coco_any.get("meta", []))  # type: ignore
    meta_by_id: Dict[int, Dict[str, Any]] = {int(m.get("id", -1)): m for m in meta_list}

    images: List[Dict[str, Any]] = list(coco_any.get("images", []))  # type: ignore
    anns: List[Dict[str, Any]] = list(coco_any.get("annotations", []))  # type: ignore

    anns_by_image: Dict[int, List[Dict[str, Any]]] = {}
    ann: Dict[str, Any]
    for ann in anns:
        img_id_val: int = int(ann.get("image_id", -1))
        anns_by_image.setdefault(img_id_val, []).append(ann)

    stage_key: str
    for stage_key in stages.keys():
        ensure_dir(out_root / stage_key / subset_name / "images")
        ensure_dir(out_root / stage_key / subset_name / "labels")

    img: Dict[str, Any]
    for img in images:
        img_id: int = int(img.get("id", -1))
        meta_id: int = int(img.get("meta_id", -1))
        file_name: str = str(img.get("file_name", ""))  # 예: "BC2000201/xxxx.jpg"

        parts: List[str] = file_name.split("/")
        camera_id: str = parts[0] if len(parts) >= 2 else ""
        fname: str = parts[-1]

        meta: Dict[str, Any] = meta_by_id.get(meta_id, {})
        weather: str = str(meta.get("weather", "")).strip()
        time_space: str = str(meta.get("time_space", "")).strip()

        stage: Optional[str] = decide_stage(weather, time_space, stages)
        if stage is None:
            counts["skipped_stage"] += 1
            continue

        src_path_opt: Optional[Path] = find_image_path_from_index(cam_index, camera_id, fname)
        if src_path_opt is None or not src_path_opt.exists():
            counts["missing_image"] += 1
            continue
        src_path: Path = src_path_opt

        dst_img_dir: Path = out_root / stage / subset_name / "images" / camera_id
        dst_lbl_dir: Path = out_root / stage / subset_name / "labels" / camera_id
        ensure_dir(dst_img_dir)
        ensure_dir(dst_lbl_dir)

        dst_img_path: Path = dst_img_dir / fname
        dst_lbl_path: Path = dst_lbl_dir / (Path(fname).stem + ".txt")

        with Image.open(src_path) as im:
            size_tuple: Tuple[int, int] = im.size
        w: int = int(size_tuple[0])
        h: int = int(size_tuple[1])

        lines: List[str] = []
        img_anns: List[Dict[str, Any]] = anns_by_image.get(img_id, [])
        ann_item: Dict[str, Any]
        for ann_item in img_anns:
            bboxes: List[List[float]] = [list(map(float, b)) for b in ann_item.get("bbox", [])]  # type: ignore
            cat_ids: List[int] = [int(c) for c in ann_item.get("category_id", [])]  # type: ignore
            idx: int
            for idx in range(0, min(len(bboxes), len(cat_ids))):
                b: List[float] = bboxes[idx]
                c: int = cat_ids[idx]
                if len(b) < 4:
                    continue
                x1: float = float(b[0]); y1: float = float(b[1]); x2: float = float(b[2]); y2: float = float(b[3])
                cxn: float; cyn: float; wn: float; hn: float
                cxn, cyn, wn, hn = xyxy_to_yolo(x1, y1, x2, y2, w, h)
                cls_idx: int = int(id_to_idx.get(c, 0))
                line: str = f"{cls_idx} {cxn:.6f} {cyn:.6f} {wn:.6f} {hn:.6f}"
                lines.append(line)

        if not dry_run:
            if OVERWRITE_LABELS or not dst_lbl_path.exists():
                with open(dst_lbl_path, "w", encoding="utf-8") as lf:
                    content: str = "\n".join(lines)
                    lf.write(content)
            if COPY_IMAGES and not dst_img_path.exists():
                shutil.copy2(src_path, dst_img_path)
            counts["labeled"] += 1
            if COPY_IMAGES:
                counts["copied"] += 1
        else:
            counts["labeled"] += 1
            if COPY_IMAGES:
                counts["copied"] += 1

    # data.yaml은 run_pipeline에서 한 번만 생성
    return counts


In [7]:
def build_category_maps(first_json_path: Path) -> Tuple[Dict[int, int], Dict[int, str]]:
    with open(first_json_path, "r", encoding="utf-8") as f0:
        first_any: Any = json.load(f0)
    categories: List[Dict[str, Any]] = list(first_any.get("categories", []))  # type: ignore

    id_to_idx: Dict[int, int] = {}
    id_to_name: Dict[int, str] = {}
    cat_obj: Dict[str, Any]
    for cat_obj in sorted(categories, key=lambda c: int(c.get("id", 0))):
        cid: int = int(cat_obj.get("id", 0))
        cname: str = str(cat_obj.get("name", ""))
        id_to_idx[cid] = cid - 1
        id_to_name[cid] = cname
    return id_to_idx, id_to_name


def _count_images_in_json(json_path: Path) -> int:
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            coco_any: Any = json.load(f)
        images_any: Any = coco_any.get("images", [])
        images: List[Dict[str, Any]] = list(images_any) if isinstance(images_any, list) else []
        return len(images)
    except Exception:
        return 0


def run_pipeline() -> Dict[str, int]:
    # 입력 유효성 확인
    if not TRAIN_LABEL_DIR.exists():
        raise FileNotFoundError(f"Missing train labels: {str(TRAIN_LABEL_DIR)}")
    if not TRAIN_IMAGE_DIR.exists():
        raise FileNotFoundError(f"Missing train images: {str(TRAIN_IMAGE_DIR)}")
    if not VAL_LABEL_DIR.exists():
        raise FileNotFoundError(f"Missing val labels: {str(VAL_LABEL_DIR)}")
    if not VAL_IMAGE_DIR.exists():
        raise FileNotFoundError(f"Missing val images: {str(VAL_IMAGE_DIR)}")

    ensure_dir(OUT_ROOT)

    # 카메라 인덱스 각각 구축
    train_cam_index: Dict[str, List[Path]] = build_camera_index(TRAIN_IMAGE_DIR)
    val_cam_index: Dict[str, List[Path]] = build_camera_index(VAL_IMAGE_DIR)

    # 카테고리 맵은 train 첫 JSON에서 추출
    train_json_files: List[Path] = sorted(TRAIN_LABEL_DIR.rglob("*.json"))
    if len(train_json_files) == 0:
        raise FileNotFoundError(f"No train JSON files under {str(TRAIN_LABEL_DIR)}")
    id_to_idx: Dict[int, int]
    id_to_name: Dict[int, str]
    id_to_idx, id_to_name = build_category_maps(train_json_files[0])

    total: Dict[str, int] = {"copied": 0, "labeled": 0, "missing_image": 0, "skipped_stage": 0}

    # 진행률 합계를 위해 JSON별 이미지 개수를 미리 계산
    train_img_counts: Dict[Path, int] = {jp: _count_images_in_json(jp) for jp in train_json_files}
    total_train_imgs: int = sum(train_img_counts.values())

    # 1) train 처리 진행바
    with tqdm(total=len(train_json_files), desc="train json files", unit="file") as pbar_files, \
         tqdm(total=total_train_imgs, desc="train images", unit="img") as pbar_imgs:
        jp: Path
        for jp in train_json_files:
            # 파일 처리
            c: Dict[str, int] = process_coco_json_fixed_subset(
                json_path=jp,
                cam_index=train_cam_index,
                out_root=OUT_ROOT,
                stages=STAGES,
                id_to_idx=id_to_idx,
                id_to_name=id_to_name,
                overwrite_labels=OVERWRITE_LABELS,
                copy_images=COPY_IMAGES,
                dry_run=DRY_RUN,
                subset_name="train"
            )
            # 카운트 갱신
            k: str
            for k in total.keys():
                total[k] += int(c.get(k, 0))
            # 진행바 업데이트
            pbar_files.update(1)
            pbar_imgs.update(train_img_counts.get(jp, 0))
            pbar_files.set_postfix({"labeled": total["labeled"], "missing": total["missing_image"], "skipped": total["skipped_stage"]})

    # 2) val 처리 준비 및 진행바
    val_json_files: List[Path] = sorted(VAL_LABEL_DIR.rglob("*.json"))
    if len(val_json_files) == 0:
        raise FileNotFoundError(f"No val JSON files under {str(VAL_LABEL_DIR)}")
    val_img_counts: Dict[Path, int] = {jp: _count_images_in_json(jp) for jp in val_json_files}
    total_val_imgs: int = sum(val_img_counts.values())

    with tqdm(total=len(val_json_files), desc="val json files", unit="file") as pbar_files, \
         tqdm(total=total_val_imgs, desc="val images", unit="img") as pbar_imgs:
        jp = Path()  # 타입 명시 목적
        for jp in val_json_files:
            c = process_coco_json_fixed_subset(
                json_path=jp,
                cam_index=val_cam_index,
                out_root=OUT_ROOT,
                stages=STAGES,
                id_to_idx=id_to_idx,
                id_to_name=id_to_name,
                overwrite_labels=OVERWRITE_LABELS,
                copy_images=COPY_IMAGES,
                dry_run=DRY_RUN,
                subset_name="val"
            )
            for k in total.keys():
                total[k] += int(c.get(k, 0))
            pbar_files.update(1)
            pbar_imgs.update(val_img_counts.get(jp, 0))
            pbar_files.set_postfix({"labeled": total["labeled"], "missing": total["missing_image"], "skipped": total["skipped_stage"]})

    # data.yaml 생성
    ordered_items: List[Tuple[int, str]] = sorted(id_to_name.items(), key=lambda kv: kv[0])
    idx_names: List[str] = [name for _, name in ordered_items]
    stage_out: str
    for stage_out in STAGES.keys():
        yaml_path: Path = OUT_ROOT / stage_out / "data.yaml"
        yaml_text: str = (
            f"path: {OUT_ROOT / stage_out}\n"
            f"train: train/images\n"
            f"val: val/images\n"
            f"nc: {len(idx_names)}\n"
            f"names: {idx_names}\n"
        )
        with open(yaml_path, "w", encoding="utf-8") as yf:
            yf.write(yaml_text)

    return total


counts_summary: Dict[str, int] = run_pipeline()
counts_summary


val json files: 100%|██████████| 46/46 [03:01<00:00,  3.94s/file, labeled=168318, missing=4, skipped=0]


{'copied': 168318, 'labeled': 168318, 'missing_image': 4, 'skipped_stage': 0}

In [8]:
summary: Dict[str, int] = counts_summary
print("Done")
print(f"copied -> {summary.get('copied', 0)}")
print(f"labeled -> {summary.get('labeled', 0)}")
print(f"missing_image -> {summary.get('missing_image', 0)}")
print(f"skipped_stage -> {summary.get('skipped_stage', 0)}")


Done
copied -> 168318
labeled -> 168318
missing_image -> 4
skipped_stage -> 0
